In [26]:
import scipy.io
import pandas as pd
import numpy as np

In [27]:
#getting train data
mat_data = scipy.io.loadmat(r"C:\Users\Dell\OneDrive\Desktop\himi\btp\dataverse_files\purified_cric1\train.mat")
mat_variable = mat_data['traindata']
data_array = np.array(mat_variable)
column_names = [f'C{i}' for i in range(1, data_array.shape[1] + 1)]
train = pd.DataFrame(data_array, columns=column_names)
train.rename(columns={'C10': 'Class_label'},inplace=True)

In [28]:
#Get test data
mat_data = scipy.io.loadmat(r"C:\Users\Dell\OneDrive\Desktop\himi\btp\dataverse_files\purified_cric1\test.mat")
mat_variable = mat_data['testdata']
data_array = np.array(mat_variable)
column_names = [f'C{i}' for i in range(1, data_array.shape[1] + 1)]
test = pd.DataFrame(data_array, columns=column_names)
test.rename(columns={'C10': 'Class_label'},inplace=True)

In [29]:
#import required libreries
from __future__ import print_function
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow import reshape
from keras.utils import np_utils
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D,AveragePooling1D
from tensorflow.keras.layers import SeparableConv1D
from tensorflow.keras.layers import ZeroPadding2D,ZeroPadding1D, MaxPooling2D, Bidirectional
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from scipy.io import loadmat
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from numpy import std
import scipy.io
from scipy import stats
from IPython.display import display, HTML
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU, SimpleRNN
from tensorflow.keras.layers import TimeDistributed
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [30]:
train["Class_label"].value_counts()

1.0    864000
2.0    864000
3.0    864000
4.0    864000
5.0    864000
6.0    864000
Name: Class_label, dtype: int64

In [31]:
train.isnull().any()

C1             False
C2             False
C3             False
C4             False
C5             False
C6             False
C7             False
C8             False
C9             False
Class_label    False
dtype: bool

In [32]:
N_FEATURES=9

In [33]:
scaler = preprocessing.StandardScaler().fit(train.iloc[:,0:N_FEATURES])

In [34]:
train.iloc[:,0:N_FEATURES]=scaler.transform(train.iloc[:,0:N_FEATURES])

In [35]:
scaler = preprocessing.StandardScaler().fit(test.iloc[:,0:N_FEATURES])
test.iloc[:,0:N_FEATURES]=scaler.transform(test.iloc[:,0:N_FEATURES])

In [36]:
train.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,Class_label
0,-0.030434,0.016899,0.019218,-0.027228,0.008080,0.008966,0.012622,-0.036836,-0.001935,1.0
1,-0.038076,0.024822,0.038545,-0.013680,0.010931,-0.010362,0.012622,-0.030883,-0.003558,1.0
2,-0.022791,0.008976,0.051429,-0.000132,0.015682,-0.029691,0.000145,-0.001118,-0.000312,1.0
3,0.000136,0.001053,0.032102,-0.000132,0.014732,-0.016805,-0.012331,0.004835,-0.000312,1.0
4,0.007778,-0.014793,0.006333,-0.006906,0.005229,0.002523,-0.012331,-0.001118,-0.000312,1.0


In [37]:
TIME_PERIODS = 1000
STEP_DISTANCE = 1000

In [38]:
def create_segments_and_labels(df, time_steps,step,n_features, label_name):
    segments = []
    labels = []
    for i in range(0, len(df), step):
      for j in range(1, n_features+1):
        L = ('C'+str(j)) 
        segments.append(df[str(L)].values[i: i + time_steps])
      label = stats.mode(df[label_name][i: i + time_steps])[0][0]
      labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [39]:
def step_decay(epoch):
   initial_lrate = 1e-3
   drop = 0.1
   epochs_drop = 70.0
   lrate = initial_lrate * tf.math.pow(drop,  
           tf.math.floor((1+epoch)/epochs_drop))
   return lrate

In [40]:
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)

In [41]:
LABEL = 'ActivityEncoded'
# Transform the labels from String to Integer via LabelEncoder
le = preprocessing.LabelEncoder()
# Add a new column to the existing DataFrame with the encoded values
train[LABEL] = le.fit_transform(train['Class_label'].values.ravel())
# df_Valid[LABEL] = le.fit_transform(df_Valid['Class_label'].values.ravel())
test[LABEL] = le.fit_transform(test['Class_label'].values.ravel())
print('df_train_size',train)
# print('df_valid_size',df_Valid)
print('df_test_size',test)

df_train_size                C1        C2        C3        C4        C5        C6        C7  \
0       -0.030434  0.016899  0.019218 -0.027228  0.008080  0.008966  0.012622   
1       -0.038076  0.024822  0.038545 -0.013680  0.010931 -0.010362  0.012622   
2       -0.022791  0.008976  0.051429 -0.000132  0.015682 -0.029691  0.000145   
3        0.000136  0.001053  0.032102 -0.000132  0.014732 -0.016805 -0.012331   
4        0.007778 -0.014793  0.006333 -0.006906  0.005229  0.002523 -0.012331   
...           ...       ...       ...       ...       ...       ...       ...   
5183995 -0.099891 -0.031499  0.006333  0.090987  0.003385  0.009771 -0.093229   
5183996 -0.054037 -0.015653  0.006333  0.080826  0.005286  0.003328 -0.211756   
5183997 -0.023467  0.000194 -0.012994  0.060503  0.003385 -0.022443 -0.230471   
5183998 -0.031110  0.023963 -0.012994  0.053729 -0.000416 -0.035329 -0.149373   
5183999 -0.038752  0.023963  0.012775  0.070665 -0.003267 -0.022443 -0.074514   

             

In [42]:
x_train, y_train = create_segments_and_labels(train,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
print('x_train shape: ', x_train.shape)
# print(x_train)
print(x_train.shape[0], 'training samples')
print('y_train shape: ', y_train.shape)
# Set input & output dimensions
num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
print(num_time_periods)
print(num_sensors)
num_classes = le.classes_.size
print(list(le.classes_))
# input_shape = (num_time_periods,num_sensors)
# print(input_shape)
input_shape = (num_time_periods,num_sensors)
#x_train = x_train.reshape(x_train.shape[0], input_shape)
print('x_train shape:', x_train[0].shape)
print('input_shape:', input_shape)
x_train = x_train.astype('float32')
# x_train = [torch.tensor(arr, dtype=torch.float32) for arr in x_train]
# y_train = y_train.astype('float32')
# print(y_train)
y_train_hot = np_utils.to_categorical(y_train, num_classes)
print(y_train_hot)
# y_train_hot= [torch.tensor(arr, dtype=torch.uint8) for arr in y_train_hot]
print('New y_train shape: ', y_train_hot.shape)

C:\Users\Dell\AppData\Local\Temp\ipykernel_21384\3477890700.py:8: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  label = stats.mode(df[label_name][i: i + time_steps])[0][0]


x_train shape:  (5184, 1000, 9)
5184 training samples
y_train shape:  (5184,)
1000
9
[1.0, 2.0, 3.0, 4.0, 5.0, 6.0]
x_train shape: (1000, 9)
input_shape: (1000, 9)
[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]]
New y_train shape:  (5184, 6)


In [43]:
x_test, y_test = create_segments_and_labels(test,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
print('x_test shape: ', x_test.shape)
# print(x_train)
print(x_test.shape[0], 'testing samples')
print('y_test shape: ', y_test.shape)
# Set input_shape / reshape for Keras
#x_test = x_test.reshape(x_test.shape[0], input_shape)
x_test = x_test.astype('float32')
y_test = y_test.astype('float32')
y_test_hot = np_utils.to_categorical(y_test, num_classes)

x_test shape:  (576, 1000, 9)
576 testing samples
y_test shape:  (576,)


C:\Users\Dell\AppData\Local\Temp\ipykernel_21384\3477890700.py:8: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  label = stats.mode(df[label_name][i: i + time_steps])[0][0]


In [44]:
n_steps, n_length = 40,25
n_depth=9
x_train = x_train.reshape(x_train.shape[0], n_steps, n_length,n_depth)
print('x_train shape: ', x_train.shape)
# x_valid = x_valid.reshape(x_valid.shape[0], n_steps, n_length, n_depth)
# print('x_valid shape: ', x_valid.shape)
x_test = x_test.reshape(x_test.shape[0], n_steps, n_length,n_depth)
print('x_test shape: ', x_test.shape)
n_outputs = y_train_hot.shape[1]
print('n_outputs',n_outputs)

x_train shape:  (5184, 40, 25, 9)
x_test shape:  (576, 40, 25, 9)
n_outputs 6


In [45]:
LABELS=[]
for i in range(1,n_outputs+1,1):
  LABELS.append (i)
print(LABELS)

[1, 2, 3, 4, 5, 6]


In [46]:
verbose, epochs, batch_size = 0, 100, 8

In [47]:
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=9, padding='same', kernel_initializer="he_normal",strides=2,kernel_regularizer=l1(1e-04)), \
                           input_shape=(n_steps,n_length,n_depth)))
model.add(TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)))
model.add(TimeDistributed(MaxPooling1D(pool_size=8,strides=2)))
model.add(TimeDistributed(Activation('tanh')))
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=5,padding="same",kernel_initializer="he_normal",strides=2,kernel_regularizer=l1(1e-04))))
model.add(TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)))
model.add(TimeDistributed(Activation('tanh')))
model.add(TimeDistributed(Dropout(0.2093)))
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=5,padding="same",kernel_initializer="he_normal",strides=2,kernel_regularizer=l1(1e-04))))
model.add(TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)))
model.add(TimeDistributed(Activation('tanh')))
model.add(TimeDistributed(Dropout(0.2093)))
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3,padding="same",kernel_initializer="he_normal",strides=2,kernel_regularizer=l1(1e-04))))
model.add(TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)))
model.add(TimeDistributed(Activation('tanh')))
model.add(TimeDistributed(Dropout(0.2093)))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Flatten()))
# model.add(Flatten())
model.add(Bidirectional(LSTM(200,return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(200,return_sequences=True)))
model.add(Dropout(0.3))
model.add(Flatten())
# model.add(Bidirectional(LSTM(200,return_sequences=True)))
# model.add(Dropout(0.3))
# model.add(Bidirectional(LSTM(200,return_sequences=True)))
# model.add(Dropout(0.3))
# model.add(Flatten())
# model.add(BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None))
# model.add(Bidirectional(LSTM(200)))
# model.add(Dropout(0.3))
model.add(Dense(2000, activation='tanh'))
model.add(Dense(512, activation='tanh'))
# model.add(Dense(200, activation='tanh'))
model.add(BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None))
model.add(Dense(n_outputs, activation='softmax'))

In [48]:
adam=optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, amsgrad=False)
checkpoint_filepath = '/media/naveen/nav/mat_codes/nina_DB2_codes/prep_no_prep/CNN100X100/checkpoint.hdf5'
# model.load_weights(checkpoint_filepath) 
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,verbose=1, monitor='val_accuracy',save_weights_only=True,save_best_only=True)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=75, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_18 (TimeDi  (None, 40, 13, 64)       5248      
 stributed)                                                      
                                                                 
 time_distributed_19 (TimeDi  (None, 40, 13, 64)       256       
 stributed)                                                      
                                                                 
 time_distributed_20 (TimeDi  (None, 40, 3, 64)        0         
 stributed)                                                      
                                                                 
 time_distributed_21 (TimeDi  (None, 40, 3, 64)        0         
 stributed)                                                      
                                                                 
 time_distributed_22 (TimeDi  (None, 40, 2, 64)       

In [49]:
LABELS=[]
for i in range(1,n_outputs+1,1):
  LABELS.append (i)
print(LABELS)

[1, 2, 3, 4, 5, 6]


In [ ]:
csv_logger = CSVLogger(r'C:\Users\Dell\OneDrive\Desktop\himi\btp\dataverse_files\purified_cric1\telidu.csv', append=True, separator=';')
history = model.fit(x_train, y_train_hot, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger,checkpoint_callback,lrate],validation_data=(x_test, y_test_hot), verbose=1)

Epoch 1/100
648/648 [==============================] - ETA: 0s - loss: 2.1337 - accuracy: 0.1590
Epoch 1: val_accuracy improved from -inf to 0.16667, saving model to /media/naveen/nav/mat_codes/nina_DB2_codes/prep_no_prep/CNN100X100\checkpoint.hdf5
648/648 [==============================] - 411s 616ms/step - loss: 2.1337 - accuracy: 0.1590 - val_loss: 2.0640 - val_accuracy: 0.1667 - lr: 0.0010
Epoch 2/100
648/648 [==============================] - ETA: 0s - loss: 2.0494 - accuracy: 0.1618
Epoch 2: val_accuracy did not improve from 0.16667
648/648 [==============================] - 373s 575ms/step - loss: 2.0494 - accuracy: 0.1618 - val_loss: 2.1735 - val_accuracy: 0.1667 - lr: 0.0010
Epoch 3/100
648/648 [==============================] - ETA: 0s - loss: 2.0079 - accuracy: 0.1570
Epoch 3: val_accuracy did not improve from 0.16667
648/648 [==============================] - 364s 562ms/step - loss: 2.0079 - accuracy: 0.1570 - val_loss: 1.9840 - val_accuracy: 0.1667 - lr: 0.0010
Epoch 4/100